In [1]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

In [2]:
import src.config as config

In [3]:
from datetime import datetime, timedelta
import pandas as pd
import pytz

# get current date rounded to hour
current_date = pd.to_datetime(datetime.now(pytz.utc)).floor("H")  # check this
print(f"{current_date=}")

fetch_data_to = current_date  # date to start fetching rides from
fetch_data_from = current_date - timedelta(days=28)  # date to end fetching rides 



current_date=Timestamp('2024-06-28 22:00:00+0000', tz='UTC')


In [12]:
# Simulate call to data warahouse
from src.data import load_raw_data

def fetch_batch_raw_data(from_date, to_date):
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    # extract data from from-date's year and months, exclude rows in rides-df whose pickup-datetime-column is after the from-date
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    # extract data from to-date's year and months, exclude rows in rides-df whose pickup-datetime-column is before the to-date
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    # combine both dataframes
    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides["pickup_datetime"] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides


In [13]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2024-05-31 22:00:00+0000', tz='UTC'), to_date_=Timestamp('2023-06-30 22:00:00+0000', tz='UTC')
[6]
File 2023-06 is already in local storage


TypeError: Invalid comparison between dtype=datetime64[us] and Timestamp

In [8]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 258/258 [00:00<00:00, 635.40it/s]


In [9]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pickup_location_id (expected type: 'bigint', derived from input: 'int') has the wrong type.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.